In [60]:
import discogs_client

In [61]:
def read_user_token():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(read_user_token().rstrip()))

In [62]:
label_ids = "1265272"
min_average_value = 4.75
min_threshold_value = 4.5

In [63]:
# COMMON FUNCTIONS

def get_labels_ids_list(label_ids):
    """ A function to convert elements to integers """
    label_id_list = [int(label_id) for label_id in label_ids.split(',')]
    return label_id_list

def get_filtered_release_lists_from_label(label, min_average, min_threshold=None):
    """ A function that returns filtered releases from label """
    all_releases = get_all_releases(label)

    filtered_release_list_1 = filter(lambda release: filter_1(release, min_average), all_releases)
    filtered_release_list_2 = filter(lambda release: filter_2(release, min_average, min_threshold), all_releases)
    
    return list(filtered_release_list_1), list(filtered_release_list_2)

def filter_1(release, min_average):
    """ A function to define a filter """
    return release.community.rating.average >= min_average and release.formats[0]['name'] == 'Vinyl' and 'Electronic' in release.genres

def filter_2(release, min_average, min_threshold):
    """ A function to define a filter """
    return min_threshold < release.community.rating.average < min_average and release.formats[0]['name'] == 'Vinyl' and 'Electronic' in release.genres

def get_all_releases(label):
    """ A function to get all releases from label """
    all_releases = []
    print()
    print("Processing label: {} {}...".format(label.name, label.url))
    for k in range(1, label.releases.pages + 1):
        print()
        print("Page: {}/{}".format(k, label.releases.pages))
        print(label.releases.page(k))
        for item in label.releases.page(k):
            all_releases.append(d.release(item.id))
    print()
    print("{} releases for label {}".format(len(all_releases), label.name))
    return all_releases

def func_year(e):
    """ A function that returns the 'year' value """
    return e.year

def remove_duplicates(input_list):
    """ A function to remove duplicates from a release input_list by id """
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = item.id
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

def get_artists(release):
    """ A function to return an artists list from release """
    artists = []
    for artist in release.artists:
        artists.append(artist.name)
    return artists

def get_labels(release):
    """ A function to return an labels list from release """
    labels = []
    for label in release.labels:
        labels.append(label.name)
    return labels

def write_output_html(label, release_list):
    """ A function to write output as HTML """

    htmlString = "<html>\n<head>\n<title>Filtered Releases for label {} - id: {}</title>\n</head>\n \
                    <body>\n \
                        <h1>Filtered Releases for label {} - id: {}</h1>\n \
                        <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                        <br> \
                        Minimum community rating average filter: <span style='color: #ff0000'>{}</span> \n \
                        <br> \
                        Number of results: {} \
                    ".format(label.name, label.id, label.name, label.id, label.url, label.url, min_average, len(release_list))

    htmlString += "<ul style='list-style: none;'>\n"
    k = 0
    for release in release_list:
        htmlString += "<br>\n"
        k += 1
        htmlString += "<span style='color: #5f5f5f'>Result {}/{}</span>\n".format(k, len(release_list)) 
        htmlString += "<li><span style='color: #9b9b9b'>catno:</span> {}</li>\n".format(release.data['labels'][0]['catno'])
        htmlString += "<li><span style='color: #9b9b9b'>Title:</span> {}</li>\n".format(release.title)
        artists = get_artists(release)
        htmlString += "<li><span style='color: #9b9b9b'>Artists:</span> {}</li>\n".format(artists)
        htmlString += "<li><span style='color: #9b9b9b'>Year:</span> {}</li>\n".format(release.year)
        labels = get_labels(release)
        htmlString += "<li><span style='color: #9b9b9b'>Labels:</span> {}</li>\n".format(labels)
        htmlString += "<li><span style='color: #9b9b9b'>Genres:</span> {}</li>\n".format(release.genres)
        htmlString += "<li><span style='color: #9b9b9b'>Styles:</span> {}</li>\n".format(release.styles)
        htmlString += "<li><span style='color: #9b9b9b'>Formats:</span> {}</li>\n".format(release.formats)
        htmlString += "<li><span style='color: #9b9b9b'>Community rating average/count:</span> <span style='color: #ff0000'>{}</span>/<span style='color: #ff8000'>{}</span></li>\n".format(release.community.rating.average, release.community.rating.count)
        htmlString += "<li><span style='color: #9b9b9b'>Url:</span> <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a></li>\n".format(release.url, release.url)

    htmlString += "</ul>\n"
    
    htmlString += "</body>\n \
                  </html>"

    file_name = "{}.html".format(label.url.lstrip('https://www.discogs.com/label/'))
    
    with open("./outputHtml/{}".format(file_name), "w") as file:
        file.write(htmlString)

    print()
    print("Written HTML output file to ./outputHtml/{}".format(file_name))

In [64]:
labels_id_list = get_labels_ids_list(label_ids)
for label_id in labels_id_list:
    result_label = d.label(label_id)
    filtered_release_list_higher_rating, filtered_release_list_middle_rating = get_filtered_release_lists_from_label(result_label, min_average_value, min_threshold_value)
    
    print()
    print("{} releases for: average community rating >= {}".format(len(filtered_release_list_higher_rating), min_average_value))
    for release in filtered_release_list_higher_rating:
        print(release)

    print()
    print("{} releases for: {} < average community rating < {}".format(len(filtered_release_list_middle_rating), min_threshold_value, min_average_value))
    for release in filtered_release_list_middle_rating:
        print(release)

    # Process filtered_release_list_higher_rating
    filtered_release_list_higher_rating.sort(reverse=True, key=func_year)
    unique_filtered_release_list_higher_rating = remove_duplicates(filtered_release_list_higher_rating)
    write_output_html(result_label, unique_filtered_release_list_higher_rating)
print()
print("Process finished")


Processing label: Artscore https://www.discogs.com/label/1265272-Artscore...

Page: 1/1
[<Release 24998860 'The Dark Welcome'>, <Release 26600165 'The Dark Welcome'>, <Release 10446578 'Diamonds & Ashes'>, <Release 10530622 'Totsuka No Tsurugi'>, <Release 11058723 'Sawmill'>, <Release 26606444 'Sawmill'>, <Release 11184384 'Anathema'>, <Release 13217748 'ARTSCORE005'>, <Release 15163690 'ARTSCORE005'>, <Release 14630885 'Hammerhead'>, <Release 13862589 'Hammerhead'>, <Release 14048028 'Onryo'>, <Release 14298828 'Die Illusion'>, <Release 15019933 'Die Illusion'>, <Release 14651640 'Time Is Relative'>, <Release 14822158 ' Time Is Relative'>, <Release 14764037 'Body Brain '>, <Release 15231419 'Wave Of Nostalgia EP '>, <Release 15949109 'Superwoman 2\u200b.\u200b0'>, <Release 16270494 'REM'>, <Release 17372029 'Cold Spots'>, <Release 17860888 'Cold Spots'>, <Release 24520745 'H O Σ T I L E '>, <Release 24700613 'H O Σ T I L E '>, <Release 24801965 'The Dark Welcome'>, <Release 24985687 